# Description

Reconcile portfolio and target positions: the ones reconstructed from fills vs ones that are logged during a prod system run.

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import logging
import os

import pandas as pd

import core.config as cconfig
import dataflow_amp.system.Cx as dtfamsysc
import helpers.hdatetime as hdateti
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hpandas as hpandas
import helpers.hprint as hprint
import im_v2.common.universe as ivcu
import oms.broker.ccxt.ccxt_aggregation_functions as obccagfu
import oms.broker.ccxt.ccxt_execution_quality as obccexqu
import oms.broker.ccxt.ccxt_logger as obcccclo
import oms.order.order_converter as oororcon
import reconciliation.sim_prod_reconciliation as rsiprrec

/app/amp/core/signal_processing/decorrelation.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-56a9d840-4ccb-49f7-9e4c-20243dcb8f31.json'
INFO  # Git
  branch_name='CmTask7790_hpandas_compare_dfs_works_differently_now'
  hash='9f0337cca'
  # Last commits:
    *   9f0337cca Samarth KaPatel Merge branch 'master' into CmTask7790_hpandas_compare_dfs_works_differently_now ( 5 minutes ago) Tue Apr 9 18:54:01 2024  (HEAD -> CmTask7790_hpandas_compare_dfs_works_differently_now, origin/CmTask7790_hpandas_compare_dfs_works_differently_now)
    |\  
    | * 038e86039 Juraj Smeriga Specify format="ISO8601" for pd.to_datetime (#7862)               (86 minutes ago) Tue Apr 9 17:33:04 2024  (origin/master, origin/HEAD, master)
    * | f5aec46fd Sameep2808 CmTask7790_hpandas_compare_dfs_works_differently_now              (   2 hours ago) Tue Apr 9 16:39:04 2024           
    |/  
# Machine info
  system=Linux
  node name=09b060c854a5
  release=5.15.0-1056-aws
  version=#61~20.04.1-

# Functions

In [3]:
# TODO(Dan): Move to a lib.
def verify_no_trades_for_missing_assets(
    reconstructed_df: pd.DataFrame,
    prod_df: pd.DataFrame,
    type_: str,
    share_asset_ids_with_no_fills: float,
) -> None:
    """
    Check that there were no executed/target trades for asset ids that are
    missing in the reconstructed df.

    :param reconstructed_df: data reconstructed from fills/parent orders
    :param prod_df: data loaded from prod run logs directly
    :param type_: data type, "portfolio" or "target_positions"
    :param share_asset_ids_with_no_fills: threshold for acceptable share
        of missing asset ids
    """
    # Set column names to check the absence of trades.
    if type_ == "portfolio":
        trades_col = "executed_trades_shares"
        shares_col = "executed_trades_notional"
    elif type_ == "target_positions":
        trades_col = "target_trades_shares"
        shares_col = "target_trades_notional"
    else:
        raise ValueError("Unsupported `type_`=%s", type_)
    missing_asset_ids = set(prod_df.columns.levels[1]) - set(
        reconstructed_df.columns.levels[1]
    )
    num_missing_asset_ids = len(missing_asset_ids)
    if num_missing_asset_ids > 0:
        # Check that the share of missing asset ids is not greater
        # than an arbitrary threshold.
        num_asset_ids = len(prod_df.columns.levels[1])
        hdbg.dassert_lte(
            num_missing_asset_ids / num_asset_ids,
            share_asset_ids_with_no_fills,
        )
        for asset_id in missing_asset_ids:
            _LOG.info("Missing asset id = %s", asset_id)
            # Check trades in shares.
            trades_shares_sum = (
                prod_df.xs(asset_id, axis=1, level=1, drop_level=False)[
                    trades_col
                ]
                .abs()
                .sum()
                .loc[asset_id]
            )
            # TODO(Dan): Consider to use `hpandas.dassert_approx_eq()`
            #  to account for a floating point error.
            hdbg.dassert_eq(
                0,
                trades_shares_sum,
                msg="Asset id='%s' with no positions/fills has executed/target trades in shares='%s' in total, while 0 is the expected value"
                % (asset_id, trades_shares_sum),
            )
            # Ideally zero trades in shares implies no notional trades,
            # but checking both in case of any data inconsistency.
            trades_notional_sum = (
                prod_df.xs(asset_id, axis=1, level=1, drop_level=False)[
                    shares_col
                ]
                .abs()
                .sum()
                .loc[asset_id]
            )
            hdbg.dassert_eq(
                0,
                trades_shares_sum,
                msg="Asset id='%s' with no with no positions/fills has executed/target nominal trades ='%s' in total, while 0 is the expected value"
                % (asset_id, trades_notional_sum),
            )


def sanity_check_indices_difference(
    reconstructed_df: pd.DataFrame,
    prod_df: pd.DataFrame,
    n_index_elements_to_ignore: int,
) -> None:
    """
    Check that the symmetric indices difference does exceed the threshold.

    :param reconstructed_df: data reconstructed from fills/parent orders
    :param prod_df: data loaded from prod run logs directly
    :param n_index_elements_to_ignore: threshold for number of differing
        indices
    """
    # Sanity check differing indices from reconstructed df.
    reconstructed_df_extra_idx = reconstructed_df.index.difference(prod_df.index)
    hdbg.dassert_lte(len(reconstructed_df_extra_idx), n_index_elements_to_ignore)
    if len(reconstructed_df_extra_idx) > 0:
        # Verify that the difference is a consecutive index.
        hpandas.dassert_strictly_increasing_index(reconstructed_df_extra_idx)
    # Sanity check differing indices from prod df.
    prod_df_extra_idx = prod_df.index.difference(reconstructed_df.index)
    hdbg.dassert_lte(len(prod_df_extra_idx), n_index_elements_to_ignore)
    if len(prod_df_extra_idx) > 0:
        # Verify that the difference is a consecutive index.
        hpandas.dassert_strictly_increasing_index(prod_df_extra_idx)

# Config

In [4]:
# When running manually, specify the path to the config to load config from file,
# for e.g., `.../reconciliation_notebook/fast/result_0/config.pkl`.
config_file_name = None
config = cconfig.get_notebook_config(config_file_name)
if config is None:
    id_col = "asset_id"
    system_log_dir = "/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts"
    # Load pickled SystemConfig.
    config_file_name = "system_config.output.values_as_strings.pkl"
    system_config_dir = system_log_dir.rstrip("/process_forecasts")
    system_config_path = os.path.join(system_config_dir, config_file_name)
    system_config = cconfig.load_config_from_pickle(system_config_path)
    # Get param values from SystemConfig.
    bar_duration_in_secs = rsiprrec.get_bar_duration_from_config(system_config)
    bar_duration = hdateti.convert_seconds_to_pandas_minutes(bar_duration_in_secs)
    universe_version = system_config["market_data_config", "universe_version"]
    price_column_name = system_config["portfolio_config", "mark_to_market_col"]
    table_name = system_config[
        "market_data_config", "im_client_config", "table_name"
    ]
    vendor = "CCXT"
    mode = "trade"
    #
    config_dict = {
        "id_col": id_col,
        "system_log_dir": system_log_dir,
        "market_data": {
            "vendor": vendor,
            "mode": mode,
            "universe_version": universe_version,
            "im_client_config": {
                "table_name": table_name,
            },
        },
        "price_column_name": price_column_name,
        "bar_duration": bar_duration,
        "share_asset_ids_with_no_fills": 0.3,
        "n_index_elements_to_ignore": 2,
        "target_positions_columns_to_compare": [
            "price",
            "holdings_shares",
            "holdings_notional",
            "target_holdings_shares",
            "target_holdings_notional",
            "target_trades_shares",
            "target_trades_notional",
        ],
        "compare_dfs_kwargs": {
            "row_mode": "inner",
            "column_mode": "inner",
            "diff_mode": "pct_change",
            "assert_diff_threshold": 1e-3,
            "log_level": logging.INFO,
        },
    }
    config = cconfig.Config.from_dict(config_dict)
print(config)

id_col: asset_id
system_log_dir: /shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts
market_data: 
  vendor: CCXT
  mode: trade
  universe_version: v8.1
  im_client_config: 
    table_name: ccxt_ohlcv_futures
price_column_name: close
bar_duration: 5T
share_asset_ids_with_no_fills: 0.3
n_index_elements_to_ignore: 2
target_positions_columns_to_compare: ['price', 'holdings_shares', 'holdings_notional', 'target_holdings_shares', 'target_holdings_notional', 'target_trades_shares', 'target_trades_notional']
compare_dfs_kwargs: 
  row_mode: inner
  column_mode: inner
  diff_mode: pct_change
  assert_diff_threshold: 0.001
  log_level: 20


# Reconstruct portfolio and target positions from fills

## Load fills

In [5]:
system_log_dir = config.get_and_mark_as_used(("system_log_dir",))
system_log_dir

'/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts'

In [6]:
ccxt_log_reader = obcccclo.CcxtLogger(system_log_dir)

WARNING Path '/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts/args.json' doesn't exist!
WARNING Path '/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts/balances' doesn't exist!
WARNING Path '/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts/reduce_only/ccxt_child_order_responses' doesn't exist!
WARNING Path '/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts/reduce_only/oms_child_orders' doesn't exist!


In [7]:
fills_df = ccxt_log_reader.load_ccxt_trades(convert_to_dataframe=True)

Loading '/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts/child…

In [8]:
fills_df.head(3)

,timestamp,datetime,symbol,asset_id,id,order,side,takerOrMaker,price,amount,cost,transaction_cost,fees_currency,realized_pnl,first_timestamp,last_timestamp,first_datetime,last_datetime,buy_count,sell_count,taker_count,maker_count,buy_volume,sell_volume,taker_volume,maker_volume,buy_notional,sell_notional,taker_notional,maker_notional
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-04-05 10:55:16.584000+00:00,2024-04-05 10:55:16.584000+00:00,2024-04-05 10:55:16.584000+00:00,BNB/USDT:USDT,8968126878,1240809621,54757624451,sell,maker,577.6000,0.170,98.1920,0.019638,USDT,0.0,2024-04-05 10:55:16.584000+00:00,2024-04-05 10:55:16.584000+00:00,2024-04-05 10:55:16.584000+00:00,2024-04-05 10:55:16.584000+00:00,0,1,0,1,0.000,0.17,0.0,0.170,0.0000,98.192,0.0,98.1920
2024-04-05 10:55:12.515000+00:00,2024-04-05 10:55:12.515000+00:00,2024-04-05 10:55:12.515000+00:00,MKR/USDT:USDT,1871387621,285252755,6931424756,buy,maker,3805.0000,0.009,34.2450,0.006849,USDT,0.0,2024-04-05 10:55:12.515000+00:00,2024-04-05 10:55:12.515000+00:00,2024-04-05 10:55:12.515000+00:00,2024-04-05 10:55:12.515000+00:00,1,0,0,1,0.009,0.00,0.0,0.009,34.2450,0.000,0.0,34.2450
2024-04-05 10:55:33.527000+00:00,2024-04-05 10:55:33.527000+00:00,2024-04-05 10:55:33.527000+00:00,IMX/USDT:USDT,1106736216,216065423,3927051012,buy,maker,2.6207,8.000,20.9656,0.004193,USDT,0.0,2024-04-05 10:55:33.527000+00:00,2024-04-05 10:55:33.527000+00:00,2024-04-05 10:55:33.527000+00:00,2024-04-05 10:55:33.527000+00:00,1,0,0,1,8.000,0.00,0.0,8.000,20.9656,0.000,0.0,20.9656


In [9]:
id_col = config.get_and_mark_as_used(("id_col",))
bar_duration = config.get_and_mark_as_used(("bar_duration",))
# Aggregate fills by bar.
bar_fills = obccagfu.aggregate_fills_by_bar(
    fills_df, bar_duration, groupby_id_col=id_col
)

In [10]:
bar_fills.head(3)

first_timestamp                   last_timestamp                   first_datetime                    last_datetime          symbol    asset_id  buy_count  sell_count  taker_count  maker_count  buy_volume  sell_volume  taker_volume  maker_volume  buy_notional  sell_notional  taker_notional  maker_notional   price  amount      cost  transaction_cost  realized_pnl
bar_end_datetime          asset_id                                                                                                                                                                                                                                                                                                                                                                                               
2024-04-05 11:00:00+00:00 1020313424 2024-04-05 10:55:33.539000+00:00 2024-04-05 10:55:33.539000+00:00 2024-04-05 10:55:33.539000+00:00 2024-04-05 10:55:33.539000+00:00    OP/USDT:USDT  1020313424          1           0            0            1        18.5          0.0           0.0          18.5      54.07365            0.0             0.0        54.07365  2.9229    18.5  54.07365          0.010815           0.0
                          1106736216 2024-04-05 10:55:33.527000+00:00 2024-04-05 10:55:33.527000+00:00 2024-04-05 10:55:33.527000+00:00 2024-04-05 10:55:33.527000+00:00   IMX/USDT:USDT  1106736216          1           0            0            1         8.0          0.0           0.0           8.0      20.96560            0.0             0.0        20.96560  2.6207     8.0  20.96560          0.004193           0.0
                          1669260535 2024-04-05 10:55:35.769000+00:00 2024-04-05 10:55:37.014000+00:00 2024-04-05 10:55:35.769000+00:00 2024-04-05 10:55:37.014000+00:00  RNDR/USDT:USDT  1669260535          2           0            0            2         7.0          0.0           0.0           7.0      62.76690            0.0             0.0        62.76690  8.9667     7.0  62.76690          0.012553           0.0

## Load oms parent orders

These are in the internal `amp` format (not the `ccxt` format)

In [11]:
parent_order_df = ccxt_log_reader.load_oms_parent_order(convert_to_dataframe=True)

Loading '/shared_data/CmTask7811/20240405_105500.20240405_115000/system_log_dir.manual/process_forecasts/oms_p…

In [12]:
parent_order_df.head(3)

,creation_timestamp,asset_id,type_,start_timestamp,end_timestamp,curr_num_shares,diff_num_shares,tz,extra_params
order_id,,,,,,,,,
0,2024-04-05 10:55:07.270150+00:00,3304664847,price@twap,2024-04-05 10:55:07.270150+00:00,2024-04-05 11:00:00+00:00,0.0,-0.114,America/New_York,"{'ccxt_symbol': 'BCH/USDT:USDT', 'stats': {'_s..."
1,2024-04-05 10:55:07.270150+00:00,8968126878,price@twap,2024-04-05 10:55:07.270150+00:00,2024-04-05 11:00:00+00:00,0.0,-0.170,America/New_York,"{'ccxt_symbol': 'BNB/USDT:USDT', 'stats': {'_s..."
2,2024-04-05 10:55:07.270150+00:00,3323452427,price@twap,2024-04-05 10:55:07.270150+00:00,2024-04-05 11:00:00+00:00,0.0,-0.600,America/New_York,"{'ccxt_symbol': 'ETC/USDT:USDT', 'stats': {'_s..."


## Load market data

In [13]:
# TODO(Paul): Refine the cuts around the first and last bars.
start_timestamp = bar_fills["first_datetime"].min() - pd.Timedelta(bar_duration)
_LOG.info("start_timestamp=%s", start_timestamp)
end_timestamp = bar_fills["last_datetime"].max() + pd.Timedelta(bar_duration)
_LOG.info("end_timestamp=%s", end_timestamp)

INFO  start_timestamp=2024-04-05 10:50:12.515000+00:00
INFO  end_timestamp=2024-04-05 11:45:55.727000+00:00


In [14]:
universe_version = config.get_and_mark_as_used(
    (
        "market_data",
        "universe_version",
    )
)
vendor = config.get_and_mark_as_used(
    (
        "market_data",
        "vendor",
    )
)
mode = config.get_and_mark_as_used(
    (
        "market_data",
        "mode",
    )
)
table_name = config.get_and_mark_as_used(
    (
        "market_data",
        "im_client_config",
        "table_name",
    )
)
# Get asset ids.
asset_ids = ivcu.get_vendor_universe_as_asset_ids(universe_version, vendor, mode)
# Get prod `MarketData`.
# TODO(Grisha): expose stage to the notebook config.
db_stage = "preprod"
market_data = dtfamsysc.get_Cx_RealTimeMarketData_prod_instance1(
    asset_ids,
    db_stage,
    table_name=table_name,
)
# Load and resample OHLCV data.
ohlcv_bars = dtfamsysc.load_and_resample_ohlcv_data(
    market_data,
    start_timestamp,
    end_timestamp,
    bar_duration,
)
ohlcv_bars.head()

INFO  Unable to fetch DB credentials from environment variables: 
	'POSTGRES_HOST'
	Attempting env file method.
INFO  Unable to fetch DB credentials from env file: 
	
################################################################################
* Failed assertion *
File '/app/amp/im_v2/devops/env/preprod.im_db_config.env' doesn't exist
################################################################################

	Attempting AWS SecretsManager method.
INFO  Fetching secret: preprod.im_data_db
INFO  Created preprod DB connection: 
 None


/app/amp/helpers/hsql_implementation.py:582: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


INFO  fit_intervals=[(Timestamp('2024-04-05 10:50:12.515000+0000', tz='UTC'), Timestamp('2024-04-05 11:45:55.727000+0000', tz='UTC'))]


close                                                                                                                                                                                                                                                                               high                                                                                                                                                                                                                                                                                low                                                                                                                                                                                                                                                                               open                                                                                                                                      \
                          1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878 1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878 1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878 1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155   
end_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
2024-04-05 06:55:00-04:00     2.9245     2.6200     8.9706     18.982     1.0308    0.32300     3803.7     81.845    171.569     128.47    0.13330     31.213     2.9536     17.156     3.0815     5.8079     687.68     33.221     1.5612     2.3903     1.4112     6.5967     16.833     44.753     577.36     2.9372     2.6300     9.0186     19.048     1.0346    0.32377     3817.1     82.097    172.049     128.68    0.13366     31.372     2.9613     17.185     3.0905     5.8571     690.76     33.356     1.5664     2.3999     1.4156     6.6311     16.880     44.926     578.19     2.9216     2.6185     8.9701     18.971     1.0301    0.32261     3802.6     81.801    171.478     128.22    0.13317     31.190     2.9517     17.145     3.0798     5.7971     687.14     33.171     1.5602     2.3899     1.4112     6.5957     16.820     44.747     576.43     2.9358     2.6275     9.0162     19.032     1.0332    0.32349     3813.8     82.018    171.925     128.63    0.13347     31.300     2.9604   
2024-04-05 07:00:00-04:00     2.9300     2.

## Compute portfolio

In [15]:
portfolio_from_fills_df = obccexqu.convert_bar_fills_to_portfolio_df(
    bar_fills,
    ohlcv_bars[config["price_column_name"]],
)
hpandas.df_to_str(portfolio_from_fills_df, num_rows=5, log_level=logging.INFO)

holdings_shares                                                                                                                                                                                                                                                              holdings_notional                                                                                                                                                                                                                                                              executed_trades_shares                                                                                                                                                                                                                                                              executed_trades_notional                                                                                                                                                                                                                                                                     pnl                                                                                                                                                                                                                                                             
asset_id                       1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878        1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878             1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878               1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878 1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878
2024-04-05 10:55:00+00:00             NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN               NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN                    NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN                      NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN       

## Compute target positions

In [16]:
target_positions_from_parent_orders_df = (
    oororcon.convert_order_df_to_target_position_df(
        parent_order_df,
        ohlcv_bars[config["price_column_name"]],
    )
)
hpandas.df_to_str(
    target_positions_from_parent_orders_df, num_rows=5, log_level=logging.INFO
)

holdings_notional                                                                                                                                                                                                                                                              holdings_shares                                                                                                                                                                                                                                                                   price                                                                                                                                                                                                                                                              target_holdings_notional                                                                                                                                                                                                                                                              target_holdings_shares                                                                                                                                                                                                                                                              target_trades_notional                                                                                                                                                                                                                                                              target_trades_shares                                                                                                                                                                                                                                                             
                                 1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878      1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878 1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878               1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878             1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878             1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878           1020313424 1106736216 1669260535 1670436375 1775391824 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878
2024-04-05 10:55:00+00:00               0.0        0.0        0.0        NaN        NaN        0.0        NaN        NaN        NaN        NaN        NaN     

# Load prod portfolio and target positions

## Portfolio

In [17]:
portfolio_dir = os.path.join(system_log_dir, "portfolio")
#
portfolio_df, _ = rsiprrec.load_portfolio_artifacts(portfolio_dir, bar_duration)
portfolio_df.head()

Loading `holdings_shares` files...:   0%|          | 0/12 [00:00<?, ?it/s]

Loading `holdings_notional` files...:   0%|          | 0/12 [00:00<?, ?it/s]

Loading `executed_trades_shares` files...:   0%|          | 0/12 [00:00<?, ?it/s]

Loading `executed_trades_notional` files...:   0%|          | 0/12 [00:00<?, ?it/s]

Loading `statistics` files...:   0%|          | 0/12 [00:00<?, ?it/s]

holdings_shares                                                                                                                                                                                                                                                                         holdings_notional                                                                                                                                                                                                                                                                         executed_trades_shares                                                                                                                                                                                                                                                                         executed_trades_notional                                                                                          \
                               2499295598 2971863517 5141755262 8717633868 3304664847 8968126878 1670436375 3323452427 1813442534 6335759645 1106736216 2506553600 4417972573 2601760471 1775391824 1871387621 1020313424 2989693852 1669260535 2237530510 2565005155 3326458212 2200033667 6009830573 2470395105        1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878             2499295598 2971863517 5141755262 8717633868 3304664847 8968126878 1670436375 3323452427 1813442534 6335759645 1106736216 2506553600 4417972573 2601760471 1775391824 1871387621 1020313424 2989693852 1669260535 2237530510 2565005155 3326458212 2200033667 6009830573 2470395105               2499295598 2971863517 5141755262 8717633868 3304664847 8968126878 1670436375 3323452427 1813442534   
2024-04-05 06:55:00-04:00             0.0        0.0        0.0        0.0      0.000       0.00        0.0       0.00        0.0        0.0        0.0        0.0        0.0       0.00        0.0      0.000        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0      0.000           0.00000     0.0000    0.00000     0.0000        0.0        0.0     0.0000     0.0000      0.000    0.00000    0.00000        0.0     0.0000    0.00000    0.00000     0.0000    0.00000    0.00000        0.0        0.0    0.00000     0.0000      0.000      0.000     0.0000                    0.0        0.0        0.0        0.0      0.000       0.00        0.0       0.00        0.0        0.0        0.0        0.0        0.0       0.00        0.0      0.000        0.0        0.0        0.0        0.0        0.0        0.0        0.0        0.0      0.000                      NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN        NaN   
2024-04-05 07:00:00-04:00             0.0        0.0        0.0        0.0     -0.114      -0.17        0.0       0.00        0.0        0.0        8.0        0.0        0.0       0.00        0.0      0.009       18.5        0.0        7.0        0.0        0.0        0.0        0.0        4.0      0.000          54.20500    21.0088   63.15260     0.0000        0.0        0.0    34.2522     0.0000      0.000    0.00000    0.00000        0.0     0.0000    0.00000    0.00000     0.0000  -78.40008    0.00000        0.0        0.0    0.00000    26.4488      0.000      0.000   -98.3399                    0.0        0.0        0.0        0.0     -0.114      -0.17        0.0       0.00        0.0        0.0        8.0        0.0        0.0       0.00        0.0      0.009       18.5        0.0        7.0        0.0        0.0        0.0        0.0        4.0      0.000                  0.00000    0.00000     0.0000      0.000  -78.27468    -98.192     0.0000    0.00000        0.0   
2024-04-05 07:05:00-04:00             0.0      -32.4        0.

## Target positions

In [18]:
target_positions_df = rsiprrec.load_target_positions(system_log_dir, bar_duration)
# Select only the columns that are necessary for reconciliation.
target_positions_df = target_positions_df[
    config["target_positions_columns_to_compare"]
]
target_positions_df.columns = target_positions_df.columns.remove_unused_levels()
#
target_positions_df.head()

Loading `target_positions` files...:   0%|          | 0/12 [00:00<?, ?it/s]

price                                                                                                                                                                                                                                                                         holdings_shares                                                                                                                                                                                                                                                                         holdings_notional                                                                                                                                                                                                                                                                         target_holdings_shares                                                                                                                \
asset_id                  1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878      1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878        1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598 2506553600 2565005155 2601760471 2971863517 2989693852 3304664847 3323452427 3326458212 4417972573 5141755262 6009830573 6335759645 8717633868 8968126878             1020313424 1106736216 1669260535 1670436375 1775391824 1813442534 1871387621 2200033667 2237530510 2470395105 2499295598   
wall_clock_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
2024-04-05 06:55:00-04:00     2.9245     2.6200     8.9706     18.982     1.0308    0.32300     3803.7     81.845    171.569     128.47    0.13330     31.213     2.9536     17.156     3.0815     5.8079     687.68     33.221     1.5612     2.3903     1.4112     6.5967     16.833     44.753     577.36             0.0        0.0        0.0        0.0        0.0        0.0      0.000        0.0        0.0      0.000        0.0        0.0        0.0       0.00        0.0        0.0      0.000       0.00        0.0        0.0        0.0        0.0        0.0        0.0       0.00           0.00000     0.0000    0.00000     0.0000        0.0        0.0     0.0000     0.0000      0.000    0.00000    0.00000        0.0     0.0000    0.00000    0.00000     0.0000    0.00000    0.00000        0.0        0.0    0.00000     0.0000      0.000      0.000     0.0000                   18.5        8.0        7.0        0.0        0.0        0.0      0.009        0.0        0.0      0.000        0.0   
2024-04-05 07:00:00-04:00     2.930

# Compare portfolio and target positions

## Portfolio

### Columns

In [19]:
portfolio_from_fills_df.columns.levels[0]

Index(['holdings_shares', 'holdings_notional', 'executed_trades_shares', 'executed_trades_notional', 'pnl'], dtype='object')

In [20]:
portfolio_df.columns.levels[0]

Index(['holdings_shares', 'holdings_notional', 'executed_trades_shares', 'executed_trades_notional', 'pnl'], dtype='object')

In [21]:
hdbg.dassert_set_eq(
    portfolio_from_fills_df.columns.levels[0],
    portfolio_df.columns.levels[0],
)

In [22]:
portfolio_from_fills_df.columns.levels[1]

Index([1020313424, 1106736216, 1669260535, 1670436375, 1775391824, 1871387621, 2200033667, 2237530510, 2470395105, 2499295598, 2506553600, 2565005155, 2601760471, 2971863517, 2989693852, 3304664847, 3323452427, 3326458212, 4417972573, 5141755262, 6009830573, 6335759645, 8717633868, 8968126878], dtype='int64', name='asset_id')

In [23]:
portfolio_df.columns.levels[1]

Index([1020313424, 1106736216, 1669260535, 1670436375, 1775391824, 1813442534, 1871387621, 2200033667, 2237530510, 2470395105, 2499295598, 2506553600, 2565005155, 2601760471, 2971863517, 2989693852, 3304664847, 3323452427, 3326458212, 4417972573, 5141755262, 6009830573, 6335759645, 8717633868, 8968126878], dtype='int64')

In [24]:
# Verify that in production portfolio all assets belong to the production universe.
hdbg.dassert_set_eq(
    asset_ids,
    portfolio_df.columns.levels[1],
)
# Having a few `asset_ids` missing in df reconstructed from fills is acceptable
# because they will be missing in case of no trades.
hdbg.dassert_set_eq(
    portfolio_from_fills_df.columns.levels[1],
    portfolio_df.columns.levels[1],
    only_warning=True,
)
# Check that for missing assets ids there were no trades.
type_ = "portfolio"
verify_no_trades_for_missing_assets(
    portfolio_from_fills_df,
    portfolio_df,
    type_,
    config["share_asset_ids_with_no_fills"],
)

WARNING * Failed assertion *
val1 - val2=[]
val2 - val1=[1813442534]
val1=24 [1020313424, ... 8968126878]
set eq
val2=25 [1020313424, ... 8968126878]
Continuing as per user request with only_warning=True
INFO  Missing asset id = 1813442534


### Index

In [25]:
# Set indices to the same timezone.
portfolio_df.index = portfolio_df.index.tz_convert("UTC")
hdbg.dassert_eq(
    portfolio_from_fills_df.index.tz,
    portfolio_df.index.tz,
)

In [26]:
# Report the symmetric difference. There are some extra consecutive index elements
# in the df reconstructed from fills that can be ignored.
hpandas.dassert_indices_equal(
    portfolio_from_fills_df,
    portfolio_df,
    only_warning=True,
)

WARNING * Failed assertion *
cond=False
df1.index.difference(df2.index)=
DatetimeIndex([], dtype='datetime64[ns, UTC]', freq=None)
df2.index.difference(df1.index)=
DatetimeIndex(['2024-04-05 11:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
Continuing as per user request with only_warning=True


In [27]:
sanity_check_indices_difference(
    portfolio_from_fills_df,
    portfolio_df,
    config["n_index_elements_to_ignore"],
)

### Values

In [28]:
# Compare values at intersecting columns / indices.
portfolio_diff_df = hpandas.compare_dfs(
    portfolio_from_fills_df, portfolio_df, **config["compare_dfs_kwargs"]
)
portfolio_diff_df.head(3)

WARNING * Failed assertion *
'0'
==
'1'
There are NaN values in one of the dataframes that are not in the other one.
Continuing as per user request with only_warning=True
INFO  Maximum percentage difference between the two dataframes = 4.440892098530698e-10


executed_trades_notional.pct_change                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           executed_trades_shares.pct_change                                                                                                                                                                                                                                                                                                                                                                                                              \
asset_id                                1020313424.pct_change 1106736216.pct_change 1669260535.pct_change 1670436375.pct_change 1775391824.pct_change 1871387621.pct_change 2200033667.pct_change 2237530510.pct_change 2470395105.pct_change 2499295598.pct_change 2506553600.pct_change 2565005155.pct_change 2601760471.pct_change 2971863517.pct_change 2989693852.pct_change 3304664847.pct_change 3323452427.pct_change 3326458212.pct_change 4417972573.pct_change 5141755262.pct_change 6009830573.pct_change 6335759645.pct_change 8717633868.pct_change 8968126878.pct_change             1020313424.pct_change 1106736216.pct_change 1669260535.pct_change 1670436375.pct_change 1775391824.pct_change 1871387621.pct_change 2200033667.pct_change 2237530510.pct_change 2470395105.pct_change 2499295598.pct_change 2506553600.pct_change 2565005155.pct_change 2601760471.pct_change 2971863517.pct_change 2989693852.pct_change 3304664847.pct_change 3323452427.pct_change 3326458212.pct_change 4417972573.pct_change   
2024-04-05 10:55:00+00:00                                 NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                               NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN   
2024-04-05 11:00:00+00:00                                 0.0                   0.0          1.132034e-14                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0          1.815511e-14                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                               0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0                   0.0          1.217350e-14                   0.0                   0.0                   0.0   
2024-04-05 11:05:00+00

## Target positions

### Columns

In [29]:
target_positions_from_parent_orders_df.columns.levels[0]

Index(['holdings_notional', 'holdings_shares', 'price', 'target_holdings_notional', 'target_holdings_shares', 'target_trades_notional', 'target_trades_shares'], dtype='object')

In [30]:
target_positions_df.columns.levels[0]

Index(['price', 'holdings_shares', 'holdings_notional', 'target_holdings_shares', 'target_holdings_notional', 'target_trades_shares', 'target_trades_notional'], dtype='object')

In [31]:
hdbg.dassert_set_eq(
    config["target_positions_columns_to_compare"],
    target_positions_from_parent_orders_df.columns.levels[0],
)
hdbg.dassert_set_eq(
    config["target_positions_columns_to_compare"],
    target_positions_df.columns.levels[0],
)

In [32]:
target_positions_df.columns.levels[1]

Index([1020313424, 1106736216, 1669260535, 1670436375, 1775391824, 1813442534, 1871387621, 2200033667, 2237530510, 2470395105, 2499295598, 2506553600, 2565005155, 2601760471, 2971863517, 2989693852, 3304664847, 3323452427, 3326458212, 4417972573, 5141755262, 6009830573, 6335759645, 8717633868, 8968126878], dtype='int64', name='asset_id')

In [33]:
target_positions_from_parent_orders_df.columns.levels[1]

Index([1020313424, 1106736216, 1669260535, 1670436375, 1775391824, 1871387621, 2200033667, 2237530510, 2470395105, 2499295598, 2506553600, 2565005155, 2601760471, 2971863517, 2989693852, 3304664847, 3323452427, 3326458212, 4417972573, 5141755262, 6009830573, 6335759645, 8717633868, 8968126878], dtype='int64')

In [34]:
# Verify that in production target positions all assets belong to the production universe.
hdbg.dassert_set_eq(
    asset_ids,
    target_positions_df.columns.levels[1],
)
# Having a few `asset_ids` missing in df reconstructed from parent orders
# is acceptablebecause they will be missing in case of no positions.
hdbg.dassert_set_eq(
    target_positions_from_parent_orders_df.columns.levels[1],
    target_positions_df.columns.levels[1],
    only_warning=True,
)
# Check that for missing assets ids there were no trades.
type_ = "target_positions"
verify_no_trades_for_missing_assets(
    target_positions_from_parent_orders_df,
    target_positions_df,
    type_,
    config["share_asset_ids_with_no_fills"],
)

WARNING * Failed assertion *
val1 - val2=[]
val2 - val1=[1813442534]
val1=24 [1020313424, ... 8968126878]
set eq
val2=25 [1020313424, ... 8968126878]
Continuing as per user request with only_warning=True
INFO  Missing asset id = 1813442534


### Index

In [35]:
# Set indices to the same timezone.
target_positions_df.index = portfolio_df.index.tz_convert("UTC")
hdbg.dassert_eq(
    target_positions_from_parent_orders_df.index.tz,
    target_positions_df.index.tz,
)

In [36]:
hpandas.dassert_indices_equal(
    target_positions_from_parent_orders_df,
    target_positions_df,
)

AssertionError: 
################################################################################
* Failed assertion *
cond=False
df1.index.difference(df2.index)=
DatetimeIndex([], dtype='datetime64[ns, UTC]', freq=None)
df2.index.difference(df1.index)=
DatetimeIndex(['2024-04-05 11:45:00+00:00', '2024-04-05 11:50:00+00:00'], dtype='datetime64[ns, UTC]', freq=None)
################################################################################


### Values

In [37]:
# Compare values at intersecting columns / indices.
target_position_diff_df = hpandas.compare_dfs(
    target_positions_from_parent_orders_df,
    target_positions_df,
    **config["compare_dfs_kwargs"],
)
target_position_diff_df.head(3)

WARNING * Failed assertion *
'0'
==
'10'
There are NaN values in one of the dataframes that are not in the other one.
Continuing as per user request with only_warning=True
INFO  Maximum percentage difference between the two dataframes = 2.4719809532670115e-12


holdings_notional.pct_change                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           holdings_shares.pct_change                                                                                                                                                                                                                                                                                                                                                                                                              \
                                 1020313424.pct_change 1106736216.pct_change 1669260535.pct_change 1670436375.pct_change 1775391824.pct_change 1871387621.pct_change 2200033667.pct_change 2237530510.pct_change 2470395105.pct_change 2499295598.pct_change 2506553600.pct_change 2565005155.pct_change 2601760471.pct_change 2971863517.pct_change 2989693852.pct_change 3304664847.pct_change 3323452427.pct_change 3326458212.pct_change 4417972573.pct_change 5141755262.pct_change 6009830573.pct_change 6335759645.pct_change 8717633868.pct_change 8968126878.pct_change      1020313424.pct_change 1106736216.pct_change 1669260535.pct_change 1670436375.pct_change 1775391824.pct_change 1871387621.pct_change 2200033667.pct_change 2237530510.pct_change 2470395105.pct_change 2499295598.pct_change 2506553600.pct_change 2565005155.pct_change 2601760471.pct_change 2971863517.pct_change 2989693852.pct_change 3304664847.pct_change 3323452427.pct_change 3326458212.pct_change 4417972573.pct_change   
end_timestamp                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2024-04-05 10:55:00+00:00                          0.0                   0.0          0.000000e+00                   NaN                   NaN                   0.0                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   0.0                   0.0                   NaN                   NaN                   NaN                   0.0                   NaN                   NaN          0.000000e+00                        0.0                   0.0                   0.0                   NaN                   NaN                   0.0                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN                   NaN          0.000000e+00                   0.0                   NaN                   NaN   
2024-04-05 11:00:00+00:00                          0.0                   0.0  